In [1]:
# prepare MEME
import pandas as pd
import numpy as np
import csv

# Build utr/orf dicts

In [2]:
filteredGeneList = pd.read_csv('../../../Database/filteredGenesDetails_human_240118.txt',sep='\t')
filteredGeneList.head(2)

,GeneName,AccNum,Chrom,mRNALength,ORFLength,fpUTR_length,tpUTR_length,sumIntronLengths
0,UBE2Q1,NM_017582,chr1,3223,1269,91,1863,6847
1,DUOXA1,NM_001276267,chr15,3032,897,395,1740,9479


In [3]:
filteredGeneList = pd.read_csv('../../../Database/filteredGenesDetails_human_240118.txt',sep='\t')
filteredGeneList.head(2)

,GeneName,AccNum,Chrom,mRNALength,ORFLength,fpUTR_length,tpUTR_length,sumIntronLengths
0,UBE2Q1,NM_017582,chr1,3223,1269,91,1863,6847
1,DUOXA1,NM_001276267,chr15,3032,897,395,1740,9479


In [35]:
acc2gene={}
for idx,row in filteredGeneList.iterrows():
    acc2gene[row['AccNum']] = row['GeneName']
acc2gene['NM_017582']

'UBE2Q1'

In [4]:
def get_fa(filename,onlyKeys = 'All'):
    keyD = dict()
    if onlyKeys!='All':
        for k in onlyKeys: keyD[k] = None
        
    def addEntry(current,d):
        entry = current.split('\n')
        if entry[0]!='' and (onlyKeys=='All' or keyD.has_key(entry[0])):
            d[entry[0]] = ''.join(entry[1:])

    d = dict()
    bf = open(filename)
    current = ['\n']
    for line in bf:
        if line[0]=='>':
            addEntry(''.join(current),d)
            if len(line)>1: current = [line[1:]]
            else: current = ['\n']
        else: current.append(line)
    addEntry(''.join(current),d)
    bf.close()
    return d

In [5]:
splicedGenes = get_fa('../../../Database/splicedGenes_240118.txt')
print(type(splicedGenes))
print(list(splicedGenes.keys())[0:3])

<class 'dict'>
['NR_148357_chr1(+):11868-14361', 'NR_046018_chr1(+):11873-14408', 'NR_024540_chr1(-):14361-29369']


In [6]:
## truncate the keys to only accNum
ks = list(splicedGenes.keys())
for key in ks:
    splicedGenes[key.split('_chr')[0]] = splicedGenes[key]
    splicedGenes.pop(key)
print(type(splicedGenes))
print(list(splicedGenes.keys())[0:3])

<class 'dict'>
['NR_148357', 'NR_046018', 'NR_024540']


In [7]:
splicedORFs = get_fa('../../../Database/splicedORFs_240118.txt')
print(type(splicedORFs))
print(list(splicedORFs.keys())[0:3])

<class 'dict'>
['NM_017582', 'NM_001276267', 'NM_173469']


In [8]:
filteredGeneList['fpUTR'] = ''
filteredGeneList['tpUTR'] = ''
filteredGeneList['ORF'] = ''
filteredGeneList.head(2)

,GeneName,AccNum,Chrom,mRNALength,ORFLength,fpUTR_length,tpUTR_length,sumIntronLengths,fpUTR,tpUTR,ORF
0,UBE2Q1,NM_017582,chr1,3223,1269,91,1863,6847,,,
1,DUOXA1,NM_001276267,chr15,3032,897,395,1740,9479,,,


In [9]:
filteredGeneList.iloc[0]['GeneName']

'UBE2Q1'

In [16]:
fpUTR={}
tpUTR = {}
ORFs = {}
for idx,row in filteredGeneList.iterrows():
    accNum = row['AccNum']
    fpLen = row['fpUTR_length']
    tpLen = row['tpUTR_length']
    fpUTR[accNum] = splicedGenes[accNum][0:fpLen]
    tpUTR[accNum] = splicedGenes[accNum][-tpLen:]
    ORFs[accNum] = splicedORFs[accNum]

In [17]:
len(filteredGeneList)

19210

In [21]:
with open('../../../Database/fpUTR.csv','w') as f:
    for key in fpUTR.keys():
        f.write("%s,%s\n"%(key,fpUTR[key]))

In [22]:
with open('../../../Database/tpUTR.csv','w') as f:
    for key in tpUTR.keys():
        f.write("%s,%s\n"%(key,tpUTR[key]))

In [23]:
with open('../../../Database/ORF.csv','w') as f:
    for key in ORFs.keys():
        f.write("%s,%s\n"%(key,ORFs[key]))    

In [24]:
!pwd

/Users/zhaoy/Documents/My Projects/RBC/Deseq2 Analysis/shift15/coding


# Extract groups of interest

In [29]:
# L24 t0
L24_t0 = pd.read_csv('csv_raw/ER_L24_t0.csv')
L24_t0_filtered = L24_t0[L24_t0['padj'] < 0.05]
print(len(L24_t0_filtered))

13


#### L24 t0:
too little, ignore first

In [30]:
L24_t0_filtered = L24_t0_filtered.sort_values(by=['log2FoldChange'])
topNum=sum((L24_t0_filtered['log2FoldChange']>1)*1)
print(topNum)
botNum=sum((L24_t0_filtered['log2FoldChange']<-1)*1)
print(botNum)
L24_t0_filtered.tail()

2
0


,Unnamed: 0,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
456,NM_033387,137.907415,0.776571,0.193303,4.017375,5.885001e-05,0.037928
3312,NM_001272068,164.297736,0.885366,0.205201,4.314619,1.598784e-05,0.027453
2679,NM_004074,715.242051,0.991900,0.244333,4.059630,4.915051e-05,0.035701
3598,NM_016633,9219.527608,1.510447,0.350191,4.313209,1.609022e-05,0.027453
7328,NM_001025604,1012.068880,3.970863,0.807518,4.917367,8.771593e-07,0.007009


In [33]:
L24_t1 = pd.read_csv('csv_raw/ER_L24_t1.csv')
L24_t1_filtered = L24_t1[L24_t1['padj'] < 0.05]
print('pass filter:' ,len(L24_t1_filtered))

L24_t1_filtered = L24_t1_filtered.sort_values(by=['log2FoldChange'])
topNum=sum((L24_t1_filtered['log2FoldChange']>1)*1)
print('up num: ',topNum)
botNum=sum((L24_t1_filtered['log2FoldChange']<-1)*1)
print('down num: ',botNum)
L24_t1_filtered.tail()

pass filter: 578
up num:  40
down num:  0


,Unnamed: 0,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
4518,NM_019554,2782.601937,1.493255,0.381964,3.909413,9.252073e-05,4.351943e-03
2679,NM_004074,715.242051,1.584397,0.245659,6.449588,1.121548e-10,2.240292e-07
689,NM_000992,359.205401,1.593513,0.336520,4.735269,2.187652e-06,3.972577e-04
4546,NM_033625,148.300167,1.625403,0.517227,3.142532,1.674935e-03,3.020933e-02
3978,NM_005694,54.781701,1.697012,0.353176,4.805007,1.547461e-06,3.253740e-04


In [47]:
def write_fa(filename,genes,orf=False):
    with open(filename,'w') as f:
        for gene in genes:
            if len(fpUTR[gene])<8:
                continue
            f.write(">%s\n%s\n"%(acc2gene[gene],fpUTR[gene]))
    

In [51]:
write_fa('../../../MOTIF/fpUTR/L24_t1_enriched.fa',list(L24_t1_filtered.tail(topNum)['Unnamed: 0']))

In [52]:
write_fa('../../../MOTIF/fpUTR/L24_t1_depleted.fa',list(L24_t1_filtered.head(topNum)['Unnamed: 0']))

In [53]:
write_fa('../../../MOTIF/fpUTR/L24_t1_notEnriched.fa',list(L24_t1_filtered.head(-topNum)['Unnamed: 0']))